In [12]:
import pandas as pd
import numpy as np

import os

import pypianoroll
import pretty_midi

import pygame

from matplotlib import pyplot as plt

In [2]:
# Directories:

data_dir = "/Users/chrisolen/Documents/uchicago_courses/deep_learning_and_image_recognition/audio_generation/data/"
scripts_dir = "/Users/chrisolen/Documents/uchicago_courses/deep_learning_and_image_recognition/audio_generation/NADE_music_generation/"

# Read In Track Metadata

In [3]:
# Read in track metadata:

track_metadata = pd.read_csv(scripts_dir+"data_cleaning_scripts/track_metadata.csv",
                            index_col='Unnamed: 0')

In [4]:
track_metadata.head()

,track_id,mb_id,artist,song_title,sp_track_id,sp_artist_id,time_sig,tempo,energy,loudness,speechiness,acousticness,instrumentalness,genre
0,TRBQUUS12903CB2580,7b28fb58-882b-4907-856e-ef327b624b11,+ / - {Plus/Minus},Flight Data Recorder,5aXSB4ywBWcPVBuOqxJAsZ,4j4NbPKY5EnsEUhdNICINb,3.0,108.021,0.584,-9.032,0.0270,0.00184,0.15500,[]
1,TRRXTJI12903D01048,6bd4eaa7-aa1c-4138-a392-41693229c7fc,+39,My Relax,2j7kfbjTPqpaYHide0hFxi,5Cicr7PrzTNv4bXiZntlex,4.0,119.997,0.553,-7.488,0.2160,0.00514,0.86900,[]
2,TRMKKFV12903CB7EA4,NaN,...arrived at ten,Flake,1T3zWGYGHIAX36IHYHL9Ar,49myFlNbBmL4X25pekwLqR,4.0,106.003,0.437,-12.814,0.0261,0.91300,0.88100,[]
3,TRNVJXQ128F14AFD63,2fc659fa-293f-4288-bff5-29447f31f628,1. Futurologischer Congress,Atem,6bsV04oYvnEEh2tmMzbuE3,6MD8yUEX0V8XJR2Xrwz0HO,4.0,136.250,0.883,-7.470,0.0513,0.03660,0.00704,['neue deutsche welle']
4,TRLIEDO12903CA41B4,f37c537b-3557-4031-bfd6-ab63ced32854,10 CC,The Things We Do For Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['nan']


In [113]:
track_metadata['loudness'].quantile(.01111111)

-26.672333823000002

In [5]:
track_metadata.shape

(31032, 14)

# Read in Midi Files

In [47]:
# Locate desired track_id from metadata:

track_id = track_metadata.iloc[800,0]

In [48]:
# Find the corresponding midi file:

midi_id = os.listdir(data_dir+"midi_files/"+ track_id)[0]

In [49]:
# Load in the full path:

mid = pypianoroll.Multitrack(data_dir+"midi_files/"+ track_id + "/" + midi_id)

In [50]:
# Is it binarized?

mid.is_binarized()

False

In [51]:
# Binarize:

mid.binarize()

In [52]:
# Store binarized int pianoroll as separate object:

roll = mid.get_stacked_pianoroll().astype(int)

In [53]:
roll.shape

(8952, 128, 19)

In [54]:
# Multitrack object metadata:

instrumentation = [mid.tracks[i].program for i in range(len(mid.tracks))]

unique, counts = np.unique(mid.tempo, return_counts=True)
tempo = dict(zip(unique, counts))

beat_res = mid.beat_resolution # number of time steps per beat

active_length = mid.get_active_length() # number of time steps where there's some activity

pitch_range = mid.get_active_pitch_range()

In [55]:
# Pianoroll object metadata (for one voice):

empty_beat_rate = pypianoroll.metrics.empty_beat_rate(roll[:,:,0], beat_res) # Ratio of empty beats to the total number of beats

num_pitches_used = pypianoroll.metrics.n_pitches_used(roll[:,:,0])

In [56]:
# Write multitrack pianroll to a MIDI file:

mid.write(scripts_dir + "modeling_scripts/"+"test_midi")


In [57]:
mid

Multitrack(tracks=["Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore", "Tormento D'amore"], tempo=array([88.00002347, 88.00002347, 88.00002347, ..., 88.00002347,
       88.00002347, 88.00002347]), downbeat=array([ True, False, False, ..., False, False, False]), beat_resolution=24, name=unknown)

# Pretty Midi

In [58]:
midi_data = pretty_midi.PrettyMIDI(data_dir+"midi_files/"+ track_id + "/" + midi_id)

In [59]:
midi_data.estimate_tempo()

179.3872105072701

In [60]:
help(midi_data)

Help on PrettyMIDI in module pretty_midi.pretty_midi object:

class PrettyMIDI(builtins.object)
 |  PrettyMIDI(midi_file=None, resolution=220, initial_tempo=120.0)
 |  
 |  A container for MIDI data in an easily-manipulable format.
 |  
 |  Parameters
 |  ----------
 |  midi_file : str or file
 |      Path or file pointer to a MIDI file.
 |      Default ``None`` which means create an empty class with the supplied
 |      values for resolution and initial tempo.
 |  resolution : int
 |      Resolution of the MIDI data, when no file is provided.
 |  intitial_tempo : float
 |      Initial tempo for the MIDI data, when no file is provided.
 |  
 |  Attributes
 |  ----------
 |  instruments : list
 |      List of :class:`pretty_midi.Instrument` objects.
 |  key_signature_changes : list
 |      List of :class:`pretty_midi.KeySignature` objects.
 |  time_signature_changes : list
 |      List of :class:`pretty_midi.TimeSignature` objects.
 |  lyrics : list
 |      List of :class:`pretty_midi.L

In [61]:
midi_data.instruments

[Instrument(program=73, is_drum=False, name="Tormento D'amore"),
 Instrument(program=47, is_drum=False, name="Tormento D'amore"),
 Instrument(program=0, is_drum=False, name="Tormento D'amore"),
 Instrument(program=40, is_drum=False, name="Tormento D'amore"),
 Instrument(program=48, is_drum=False, name="Tormento D'amore"),
 Instrument(program=49, is_drum=False, name="Tormento D'amore"),
 Instrument(program=48, is_drum=False, name="Tormento D'amore"),
 Instrument(program=9, is_drum=False, name="Tormento D'amore"),
 Instrument(program=32, is_drum=False, name="Tormento D'amore"),
 Instrument(program=45, is_drum=False, name="Tormento D'amore"),
 Instrument(program=57, is_drum=False, name="Tormento D'amore"),
 Instrument(program=71, is_drum=False, name="Tormento D'amore"),
 Instrument(program=48, is_drum=True, name="Tormento D'amore"),
 Instrument(program=72, is_drum=False, name="Tormento D'amore"),
 Instrument(program=60, is_drum=False, name="Tormento D'amore"),
 Instrument(program=52, is_d

In [62]:
midi_data.key_signature_changes

[KeySignature(key_number=0, time=0.0),
 KeySignature(key_number=2, time=190.90904)]

In [63]:
midi_data.time_signature_changes

[TimeSignature(numerator=4, denominator=4, time=0.0)]

In [64]:
midi_data.get_beats()[0:5]

array([0.      , 0.681818, 1.363636, 2.045454, 2.727272])

In [65]:
midi_data.get_chroma().shape

(12, 25363)

In [66]:
midi_data.get_piano_roll().shape

(128, 25363)

In [67]:
midi_data.estimate_beat_start()

5.4772712666666665

In [68]:
midi_data.get_downbeats()[0:5]

array([ 0.      ,  2.727272,  5.454544,  8.181816, 10.909088])

In [69]:
midi_data.get_pitch_class_histogram()

array([0.15712427, 0.017892  , 0.14996747, 0.        , 0.1655823 ,
       0.02179571, 0.07937541, 0.17208848, 0.        , 0.13630449,
       0.        , 0.09986988])

In [70]:
midi_data.__dict__

{'resolution': 120,
 '_tick_scales': [(0, 0.0056818166666666664)],
 '_PrettyMIDI__tick_to_time': array([0.00000000e+00, 5.68181667e-03, 1.13636333e-02, ...,
        2.53630614e+02, 2.53636296e+02, 2.53641978e+02]),
 'key_signature_changes': [KeySignature(key_number=0, time=0.0),
  KeySignature(key_number=2, time=190.90904)],
 'time_signature_changes': [TimeSignature(numerator=4, denominator=4, time=0.0)],
 'lyrics': [Lyric(text="C= ", time=2.045454), Lyric(text="
  ", time=2.045454), Lyric(text="Mar", time=26.931811), Lyric(text="ti", time=27.27272), Lyric(text="nio ", time=27.613629), Lyric(text="tor", time=27.954538), Lyric(text="men", time=28.295447), Lyric(text="to ", time=28.977265), Lyric(text="d'a", time=29.659083), Lyric(text="mo", time=29.999992), Lyric(text="re ", time=30.68181), Lyric(text="
  ", time=30.68181), Lyric(text="Il ", time=32.386355), Lyric(text="de", time=32.727264), Lyric(text="lirio ", time=33.068173), Lyric(text="fa ", time=33.409082), Lyric(text="par", time=

In [71]:
midi_data.time_to_tick(time=1)

176

In [102]:
midi_data.get_piano_roll()[30:90,2000:2015]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,

# Playing Midi File in Python

In [88]:
pygame.mixer.init()
pygame.mixer.music.load(data_dir+"midi_files/"+ track_id + "/" + midi_id)
pygame.mixer.music.play()

In [90]:
pygame.mixer.music.stop()

# Plotting